In [ ]:
import os
import logging
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import optuna
import tensorflow as tf
from sklearn.metrics import accuracy_score, precision_recall_curve, average_precision_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import confusion_matrix, roc_curve, auc
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from dataclasses import dataclass
import sys
sys.path.append('C:\\Users\\Aeesha\\DissProject\\mlbearing')
import time
from datetime import datetime
import scipy
from scipy.stats import entropy
from dataclasses import dataclass
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,f1_score,classification_report
from sklearn import preprocessing
from src.exception import CustomException
from src.logger import logging
from src.utils import save_object
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
import logging
import sys

In [ ]:

@dataclass
# class ModelTrainerConfig:
#     trained_model_file_path: str = os.path.join("artifacts", "model.pkl")

class ModelTrainer:
    def __init__(self):
        # self.model_trainer_config = ModelTrainerConfig()
        self.B1, self.B2, self.B3, self.B4 = self._initialize_dicts()

    def _initialize_dicts(self):
        B1 ={
            "early" : ["2003-10-22 12:06:24" , "2003-10-23 09:14:13"],
            "suspect" : ["2003-10-23 09:24:13" , "2003-11-08 12:11:44"],
            "normal" : ["2003-11-08 12:21:44" , "2003-11-19 21:06:07"],
            "suspect_1" : ["2003-11-19 21:16:07" , "2003-11-24 20:47:32"],
            "imminent_failure" : ["2003-11-24 20:57:32","2003-11-25 23:39:56"]
        }
        B2 = {
            "early" : ["2003-10-22 12:06:24" , "2003-11-01 21:41:44"],
            "normal" : ["2003-11-01 21:51:44" , "2003-11-24 01:01:24"],
            "suspect" : ["2003-11-24 01:11:24" , "2003-11-25 10:47:32"],
            "imminient_failure" : ["2003-11-25 10:57:32" , "2003-11-25 23:39:56"]
        }

        B3 = {
            "early" : ["2003-10-22 12:06:24" , "2003-11-01 21:41:44"],
            "normal" : ["2003-11-01 21:51:44" , "2003-11-22 09:16:56"],
            "suspect" : ["2003-11-22 09:26:56" , "2003-11-25 10:47:32"],
            "Inner_race_failure" : ["2003-11-25 10:57:32" , "2003-11-25 23:39:56"]
        }

        B4 = {
            "early" : ["2003-10-22 12:06:24" , "2003-10-29 21:39:46"],
            "normal" : ["2003-10-29 21:49:46" , "2003-11-15 05:08:46"],
            "suspect" : ["2003-11-15 05:18:46" , "2003-11-18 19:12:30"],
            "Rolling_element_failure" : ["2003-11-19 09:06:09" , "2003-11-22 17:36:56"],
            "Stage_two_failure" : ["2003-11-22 17:46:56" , "2003-11-25 23:39:56"]
        }  # the B4 dictionary
        
        return B1, B2, B3, B4

    def _get_state(self, time, dates_dict):
        # Convert string time to datetime object for comparison
        time = datetime.strptime(time, '%Y-%m-%d %H:%M:%S')
        for state, date_ranges in dates_dict.items():
            start_date = datetime.strptime(date_ranges[0], '%Y-%m-%d %H:%M:%S')
            end_date = datetime.strptime(date_ranges[1], '%Y-%m-%d %H:%M:%S')
            
            # Check if the given time falls between start_date and end_date
            if start_date <= time <= end_date:
                return state
        return None



    def _assign_states(self, df):
        for col, dates_dict in zip(["B1_state", "B2_state", "B3_state", "B4_state"], [self.B1, self.B2, self.B3, self.B4]):
            df[col] = df.index.map(lambda x: self._get_state(x, dates_dict))
                    
        return df

    def initiate_model_trainer(self):
        try:
            logging.info("Split training and test input data")
            df = pd.read_csv('./notebook/data/set1_timefeatures.csv')
            set1 = df.rename(columns={'Unnamed: 0':'time'}).set_index('time')
            set1 = self._assign_states(set1)
            
            # Refactoring the loop for code reusability
            state_configs = {
                "B1_state": [(151, "early"), (600, "suspect"), (1499, "normal"), (2098, "suspect"), (2156, "imminent_failure")],
                "B2_state": [(500, "early"), (2000, "normal"), (2120, "suspect"), (2156, "imminent_failure")],
                "B3_state": [(500, "early"), (1790, "normal"), (2120, "suspect"), (2156, "Inner_race_failure")],
                "B4_state": [(200, "early"), (1000, "normal"), (1435, "suspect"), (1840, "Inner_race_failure"), (2156, "Stage_two_failure")]
            }

            # Initializing states for each column based on counter
            for state_col, config in state_configs.items():
                state_list = list()
                for limit, state_name in config:
                    state_list.extend([state_name] * (limit - len(state_list)))
                set1[state_col] = state_list + [None] * (len(set1) - len(state_list))

            # Handle None values, you can choose to replace them or perform any other operation
            set1 = set1.fillna(method='ffill')

                            
            B1_cols = [col for col in set1.columns if "B1" in col]
            B2_cols = [col for col in set1.columns if "B2" in col]
            B3_cols = [col for col in set1.columns if "B3" in col]
            B4_cols = [col for col in set1.columns if "B4" in col]

            B1 = set1[B1_cols]
            B2 = set1[B2_cols]
            B3 = set1[B3_cols]
            B4 = set1[B4_cols]
            cols = ['Bx_mean','Bx_std','Bx_skew','Bx_kurtosis','Bx_entropy','Bx_rms','Bx_max','Bx_p2p','Bx_crest', 'Bx_clearence', 'Bx_shape', 'Bx_impulse',
                        'By_mean','By_std','By_skew','By_kurtosis','By_entropy','By_rms','By_max','By_p2p','By_crest', 'By_clearence', 'By_shape', 'By_impulse',
                        'class']
            # cols = [... ]  # The same list of column names as before
            B1.columns = cols
            B2.columns = cols
            B3.columns = cols
            B4.columns = cols
            final_data = pd.concat([B1, B2, B3, B4], axis=0, ignore_index=True)

            X = final_data.copy()
            y = X.pop("class")
            le = preprocessing.LabelEncoder()
            y = le.fit_transform(y)

            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)
            
            # Reshape the data for LSTM
            X_train = np.reshape(X_train.values, (X_train.shape[0], 1, X_train.shape[1]))
            X_test = np.reshape(X_test.values, (X_test.shape[0], 1, X_test.shape[1]))

            return(X_train, X_test, y_train, y_test)

        except Exception as e:
            raise CustomException(str(e))

        # For RNN 
        # Modifying the objective_lstm function

    def objective_lstm(self, trial):
        start_time = time.time()
        
        X_train, X_test, y_train, y_test = self.initiate_model_trainer()

        n_units = trial.suggest_int("n_units", 16, 128, log=True)
        dropout_rate = trial.suggest_float("dropout_rate", 0.1, 0.5, step=0.1)
        learning_rate = trial.suggest_float("learning_rate", 1e-4, 1e-2, log=True)

        model = tf.keras.Sequential([
            tf.keras.layers.LSTM(units=n_units, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])),
            tf.keras.layers.Dropout(rate=dropout_rate),
            tf.keras.layers.LSTM(units=n_units),
            tf.keras.layers.Dropout(rate=dropout_rate),
            tf.keras.layers.Dense(units=len(np.unique(y_train)), activation='softmax')
        ])

        model.compile(
            loss="sparse_categorical_crossentropy",
            optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
            metrics=["accuracy"]
        )

        model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2, verbose=0)

        y_pred = model.predict(X_test)
        y_pred_classes = np.argmax(y_pred, axis=1)

        accuracy = accuracy_score(y_test, y_pred_classes)

        end_time = time.time()
        self.log_and_print("LSTM", accuracy, end_time - start_time)

        # Compute ROC curve and ROC area
        fpr, tpr, _ = roc_curve(y_test, y_pred)
        roc_auc = auc(fpr, tpr)

        # Plot
        plt.figure()
        plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
        plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('Receiver Operating Characteristic (ROC) Curve')
        plt.legend(loc="lower right")
        plt.show()

        # Compute Precision-Recall curve
        precision, recall, _ = precision_recall_curve(y_test, y_pred)
        avg_precision = average_precision_score(y_test, y_pred)

        # Plot
        plt.figure()
        plt.plot(recall, precision, color='b', lw=2, label='Precision-Recall curve (area = %0.2f)' % avg_precision)
        plt.fill_between(recall, precision, step='post', alpha=0.2, color='b')
        plt.xlabel('Recall')
        plt.ylabel('Precision')
        plt.title('Precision-Recall Curve')
        plt.legend(loc="upper right")
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.show()

        return accuracy

    def log_and_print(self, model_name, score, duration):
        print(f"Model {model_name} found with score: {score:.4f}")
        print(f"Model {model_name} took {duration:.2f} seconds to train.")


In [ ]:
vibd = ModelTrainer()

# Optimize for LSTM
study_lstm = optuna.create_study(direction='maximize')
study_lstm.optimize(vibd.objective_lstm, n_trials=20)  # 20 trials, adjust this as needed

print("\n--- LSTM Results ---")
print("Best Parameters: ", study_lstm.best_params)
print("Best Score: ", study_lstm.best_value)
    